In [ ]:
import requests
import re
import tabula
import pandas as pd

# Download of Module Handbooks

In [ ]:
pdf_url_bachelor_business_management_and_economics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-184-H-2008.pdf"
pdf_url_bachelor_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-277-H-2021.pdf"
pdf_url_bachelor_economathematics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-276-H-2015.pdf"
pdf_url_master_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-j10-H-2019.pdf"
pdf_url_master_management ="https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-i45-H-2018.pdf"
pdf_url_master_international_economic_policy = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-h92-H-2018.pdf"

response = requests.get(pdf_url_bachelor_business_management_and_economics)
with open("module_handbooks/bachelor_business_management_and_economics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_business_management_and_economics.pdf")

response = requests.get(pdf_url_bachelor_information_systems)
with open("module_handbooks/bachelor_information_systems.pdf", "wb") as f:
    f.write(response.content)

print("Downloaded bachelor_information_systems.pdf")
response = requests.get(pdf_url_bachelor_economathematics)
with open("module_handbooks/bachelor_economathematics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_economathematics.pdf")

response = requests.get(pdf_url_master_information_systems)
with open("module_handbooks/master_information_systems.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_information_systems.pdf")

response = requests.get(pdf_url_master_management)
with open("module_handbooks/master_management.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_management.pdf")

response = requests.get(pdf_url_master_international_economic_policy)
with open("module_handbooks/master_international_economic_policy.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_international_economic_policy.pdf")

# Ansatz über TABULA --> Verforfen 
Reading Tables from Module Book for master_information_systems

In [ ]:
import pandas as pd
import tabula
import re

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:


tables_master_is = tabula.read_pdf("module_handbooks/master_information_systems.pdf", pages='10-30', multiple_tables=True)
print(len(tables_master_is))
print(tables_master_is)

# PDF Miner Ansatz wie Felix -> über Regex

In [ ]:
import io
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage


def pdf_to_text(file_path, n=0):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()

    laparams = LAParams()
    laparams.line_margin = 2.1
    laparams.word_margin = 0.1
    laparams.boxes_flow = 0.01
    # laparams.detect_vertical = True
    # laparams.all_texts = True


    converter = TextConverter(
        resource_manager,
        fake_file_handle,
        codec="utf-8",
        laparams=laparams,
    )
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(file_path, "rb") as fh:
        for i, page in enumerate(PDFPage.get_pages(fh, caching=True, check_extractable=True)):
            if i < n:
                continue
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    converter.close()
    fake_file_handle.close()

    if text:
        return remove_header_footer(text)
    else:
        return ""


def remove_header_footer(text):
    # Split the text into lines
    lines = text.split("\n")

    # Remove lines that match the header pattern
    lines = [line for line in lines if not line.startswith("JMU Würzburg")]

    # Remove lines that match the footer pattern
    lines = [line for line in lines if not line.startswith("page")]
    lines = [line for line in lines if not re.match(r".*Master.* with 1 major.*", line)]
    lines = [line for line in lines if not re.match(r".*Master.* - 2019.*", line)]
    lines = [line for line in lines if not re.match(r".*Module Catalogue for the Subject.*", line)]
    lines = [line for line in lines if not re.match(r".*Information Systems.*", line)]
    lines = [line for line in lines if not re.match(r".*Master’s with 1 major, 120 ECTS credits.*", line)]


    # Join the remaining lines back into a single string
    clean_text = "\n".join(lines)

    return clean_text

In [ ]:
text = pdf_to_text("module_handbooks/master_information_systems.pdf", 9)

In [ ]:
import io
import PyPDF2

def pdf_to_text_pypdf(file_path):
    text = ""
    with open(file_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)  # Use PdfReader instead of PdfFileReader
        num_pages = len(pdf_reader.pages)
        for page in range(num_pages):
            page_obj = pdf_reader.pages[page]
            text += page_obj.extract_text()
    return text


text = pdf_to_text_pypdf("module_handbooks/master_information_systems.pdf")

In [ ]:
text_no_footer_header = remove_header_footer(text)
print(text_no_footer_header)

with open("module_handbooks/master_information_systems_processed.txt", "w") as f:
    f.write(text_no_footer_header)

In [ ]:
tables = re.split(r"(?=Module title)", text_no_footer_header)[1:]
print(len(tables))
print("---------------TABLE 0--------------------------")
print(tables[2])
# store to table.txt file 
with open("module_handbooks/master_information_systems_table_0.txt", "w") as f:
    f.write(tables[0])

with open("module_handbooks/master_information_systems_table_1.txt", "w") as f:
    f.write(tables[1])

In [ ]:
def extract_table_values(table_string):
    patterns = {
        "Module title": r"Module title(.*?)\sModule coordinator",
        "Abbreviation": r"Abbreviation(.*?)\s+Module offered by",
        "Method of grading": r"grading(.*?)\s\d",
        "ECTS": r"numerical grade*(.*?)\sModule level",
        "Contents": r"Contents\s+(.*?)Intended learning outcomes",
        "Intended learning outcomes": r"Intended learning outcomes\s+(.*?)\s+Courses",
        "Methods of assessment": "Method of assessment  \(type, scope, language — if other than German, examination offered — if not every semester, information on whether module is creditable for bonus\)\s+(.*?)\s+Allocation",
        "Workload": r"Workload\s+(.*?)\s+Teaching cycle",
        "Teaching cycle": r"Teaching cycle\s+(.*?)\s+Referred to in LPO I",
        "Referred to in LPO I": r"Referred to in LPO I  \(examination regulations for teaching-degree programmes\)+(.*?)Module appears in",
        "Courses": r"German\)(.*?)Method",
        "Module appears in": r"Module appears in\s(.*?)\sAbbreviation",
        "Addititional information": r"Additional information\s+(.*?)\s+Workload",
        "Allocation of places": r"Allocation of places\s+(.*?)\s+Additional information",
        "Only after succ. compl. of module(s)":r"(?<=ECTS\s)([^\n]+)\s+Other prerequisites",
        "Module coordinator": r"Module coordinator(.*?)\s+ECTS",
        # "Duration": r"Duration\s+(\d\ssemester|1\ssemester)\s",
        "Module offered by": r"Module offered by(.*?)Only after succ",
    }   

    extracted_values = {}

    # convert the pattern.items() to lowercase
    patterns = {key.lower(): value for key, value in patterns.items()}

    # Extract values using regex
    for key, pattern in patterns.items():
        match = re.search(pattern, table_string, re.DOTALL)
        if match:
            extracted_values[key] = match.group(1).strip()

    return extracted_values

In [ ]:
print("---------------TABLE 0--------------------------")
# print(tables[0])
values = extract_table_values(tables[0])
# Print the extracted values
for key, value in values.items():
    print(key + ": " + value)


print("\n---------------TABLE 1--------------------------")
values = extract_table_values(tables[1])
for key, value in values.items():
    print(key + ": " + value)

print("\n---------------TABLE 2--------------------------")
values = extract_table_values(tables[2])
for key, value in values.items():
    print(key + ": " + value)

print("\n---------------TABLE 3--------------------------")
values = extract_table_values(tables[3])
for key, value in values.items():
    print(key + ": " + value)

# Create final Dataframe with these columns

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


def process_module_handbook(tables):
    module_handbook_columns = [
        "Module title",
        "Abbreviation",
        "Module coordinator",
        "Module offered by",
        "ECTS",
        "Method of grading",
        "Duration",
        "Module Level",
        "Other Prerequisites",
        "Contents",
        "Intended Learning Outcomes",
        "Courses",
        "Method of Assessment",
        "Allocation of Places",
        "Additional Information",
        "Workload",
        "Teaching Cycle",
        "Referred to in LPO I",
        "Module appears in",
    ]
    # convert to lowercase
    module_handbook_columns = [column.lower() for column in module_handbook_columns]
    module_handbook_df = pd.DataFrame(columns=module_handbook_columns)

    for table in tables:
        values = extract_table_values(table)
        # Append the values as a new row to the module_handbook_df dataframe
        module_handbook_df = module_handbook_df.append(values, ignore_index=True)

    return module_handbook_df


df_compl = process_module_handbook(tables)
# store to csv file and excel
df_compl.to_csv("module_handbooks/master_information_systems.csv", index=False)


the preprocessing step

In [ ]:
def process_module_handbook(file_path):
    text = pdf_to_text(file_path, n=0)
    print("Done with converting to string")

    text_no_footer_header = remove_header_footer(text)
    print("Done with removing header and footer")

    tables = re.split(r"(?=Module title)", text_no_footer_header)[1:]
    print("Done with splitting into tables")

    module_handbook_columns = [
        "Module title",
        "Abbreviation",
        "Module coordinator",
        "Module offered by",
        "ECTS",
        "Method of grading",
        "Completion of Modules",
        "Duration",
        "Module Level",
        "Other Prerequisites",
        "Contents",
        "Intended Learning Outcomes",
        "Courses",
        "Method of Assessment",
        "Allocation of Places",
        "Additional Information",
        "Workload",
        "Teaching Cycle",
        "Referred to in LPO I",
        "Module appears in",
    ]
    # convert to lowercase
    module_handbook_columns = [column.lower() for column in module_handbook_columns]
    module_handbook_df = pd.DataFrame(columns=module_handbook_columns)

    for table in tables:
        values = extract_table_values(table)
        # Append the values as a new row to the module_handbook_df dataframe
        module_handbook_df = module_handbook_df.append(values, ignore_index=True)

    return module_handbook_df

df_compl = process_module_handbook("module_handbooks/master_information_systems.pdf")

In [ ]:
df_compl = process_module_handbook("module_handbooks/master_management.pdf")

In [358]:
print(df_compl.shape)

# in column appears in, replace the \n with a comma 
df_compl["module appears in"] = df_compl["module appears in"].str.replace("\n", ", ")
# trim all strings (that means remove leading and trailing whitespaces)
df_compl = df_compl.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# what is the value of "module title=IT management" 
df_compl.to_csv("module_handbooks/master_management.csv", index=False)
print(df_compl["abbreviation"].value_counts())


(499, 21)
12-M-OE1-182-m01       8
12-M-KOBO-182-m01      8
12-M-SDC-182-m01       8
12-M-MADM-182-m01      8
12-M-EC1-182-m01       7
                      ..
12-M-MS-161-m01        1
12-M-POPIM2-182-m01    1
12-M-POPIM1-182-m01    1
12-M-RTP-182-m01       1
12-M-THB-161-m01       1
Name: abbreviation, Length: 187, dtype: int64
